# [PACKUS] 유저별 총 주문금액, Active User 수, 1인당 주문 금액 (1)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data 1) SKT 배달음식 + 강수량 + (NICE)편의점 3종 음식

In [2]:
new = pd.read_csv('new_data.csv')

In [3]:
new.columns = ['날짜','족발/보쌈','중국음식','치킨','피자','일강수량(mm)','컵라면','김밥','도시락']

In [4]:
new['날짜'] = pd.to_datetime(new['날짜']).dt.to_period('D')

### Data2) (패커스) 총 주문 금액( = y값 )

In [5]:
order = pd.read_excel('(통합)고객 주문리스트17-19.xlsx')

In [7]:
order['날짜'] = pd.to_datetime(order['결제일시(입금확인일)'], errors='coerce').dt.to_period('D')

In [8]:
packus = pd.DataFrame(order.groupby('날짜').sum()['총 주문금액'])
packus = packus.reset_index()

### Data3) (패커스) 시간별 누적 회원 가입자 수

In [9]:
user = pd.read_excel('(통합)DB분석_회원관리서버.xlsx')
user['최종접속일'] = pd.to_datetime(user['최종접속일']).dt.to_period('D')
user['회원 가입일'] = pd.to_datetime(user['회원 가입일']).dt.to_period('D')
user['최종주문일'] = pd.to_datetime(user['최종주문일']).dt.to_period('D')
user['최종로그인일'] = pd.to_datetime(user['최종로그인일'], errors='coerce').dt.to_period('D')
user['최종로그인일'] = user['최종로그인일'].dt.to_timestamp()
user['휴면해제일'] = pd.to_datetime(user['휴면해제일'], errors='coerce').dt.to_period('D')
user['휴면해제일'] = user['휴면해제일'].dt.to_timestamp()
emailuser = user['아이디'].str.extract(r'(\w+)@(.+)').dropna()
indices = emailuser.index
user.loc[indices,'아이디'] = emailuser[0]

realfake = user[user.duplicated('아이디', keep=False)].sort_values('아이디').index

dupuser = user[user.duplicated('아이디', keep=False)].sort_values('아이디')[['아이디','최종로그인일']].dropna()
dupuser = dupuser[dupuser['최종로그인일']>pd.to_datetime('2010-01-01')]
dupuser = dupuser.sort_values('최종로그인일').drop_duplicates('아이디', keep='last')
dupindex = dupuser.index

realindex = user.index.difference(realfake).union(dupindex)

C:\Users\samsung\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [10]:
realuser = user.ix[realindex]
realuser = realuser[['아이디','회원 가입일','최종주문일']]

C:\Users\samsung\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [11]:
makeid = pd.DataFrame(realuser['회원 가입일'].value_counts())
makeid = makeid.reset_index()
makeid['index'] = pd.to_datetime(makeid['index'].astype('str'))

In [12]:
makeid2 = pd.DataFrame(realuser['최종주문일'].value_counts())
makeid2 = makeid2.reset_index()
makeid2['index'] = pd.to_datetime(makeid2['index'].astype('str'))

In [13]:
ID = pd.merge(makeid,makeid2,how='outer',left_on=['index'], right_on=['index'])

### Merge Data 1,2,3
- Data1 : (1) SKT 업종별 전화 배달/ (2) 기상청 강수량 / (3) NICE 3종음식 편의점
- Data2 : (4) 패커스 총 주문금액
- Data3 : (5) 패커스 IN & OUT 사람 수 ( by 회원가입일 & 최종주문일 )

In [14]:
new.head()

,날짜,족발/보쌈,중국음식,치킨,피자,일강수량(mm),컵라면,김밥,도시락
0,2013-08-01,2989.0,29310.0,21906.0,6624.0,0.0,NaN,NaN,NaN
1,2013-08-02,3389.0,30199.0,27457.0,8091.0,7.5,NaN,NaN,NaN
2,2013-08-03,3935.0,35965.0,32984.0,9601.0,0.1,NaN,NaN,NaN
3,2013-08-04,3682.0,36219.0,31856.0,10519.0,4.5,NaN,NaN,NaN
4,2013-08-05,2645.0,27561.0,19661.0,6318.0,7.0,NaN,NaN,NaN


In [15]:
packus.head()

,날짜,총 주문금액
0,2017-01-02,70500
1,2017-01-15,21000
2,2017-01-18,77700
3,2017-01-23,21000
4,2017-02-02,76000


In [16]:
ID.head()

,index,회원 가입일,최종주문일
0,2018-10-25,64.0,29.0
1,2018-10-23,61.0,33.0
2,2019-01-28,56.0,37.0
3,2018-10-24,55.0,19.0
4,2018-05-14,54.0,18.0


### data 1 + 2

In [17]:
data = pd.merge(new,packus, how='outer',left_on=['날짜'],right_on=['날짜'])
data['날짜'] = data['날짜'].astype('str')
ID['index'] = ID['index'].astype('str')

### data 1 + 2 + 3

In [18]:
Real = pd.merge(data,ID, how='outer',left_on=['날짜'],right_on=['index']).drop('index',axis=1)
Real['날짜'] = pd.to_datetime(Real['날짜'].astype('str'))
Real.head()

,날짜,족발/보쌈,중국음식,치킨,피자,일강수량(mm),컵라면,김밥,도시락,총 주문금액,회원 가입일,최종주문일
0,2013-08-01,2989.0,29310.0,21906.0,6624.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-08-02,3389.0,30199.0,27457.0,8091.0,7.5,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-08-03,3935.0,35965.0,32984.0,9601.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-08-04,3682.0,36219.0,31856.0,10519.0,4.5,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-08-05,2645.0,27561.0,19661.0,6318.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN


### DATA 별로, 기간이 다름.
> 1) 2017.01.01~ 2018.12.31 : 완전한 data

> 2) 2017.01.01~ 2019.08.31 : (NICE 편의점 data값이 없는) 불완전한 data

In [19]:
modeldata = Real.copy()
modeldata = modeldata.fillna(0) 

In [20]:
modeldata['누적 가입자수'] = modeldata['회원 가입일'].cumsum()
modeldata['누적 이탈자수'] = modeldata['최종주문일'].cumsum()
modeldata.drop(['회원 가입일','최종주문일'],axis=1,inplace=True)

In [23]:
modeldata.head()

,날짜,족발/보쌈,중국음식,치킨,피자,일강수량(mm),컵라면,김밥,도시락,총 주문금액,누적 가입자수,누적 이탈자수
0,2013-08-01,2989.0,29310.0,21906.0,6624.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-08-02,3389.0,30199.0,27457.0,8091.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-08-03,3935.0,35965.0,32984.0,9601.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-08-04,3682.0,36219.0,31856.0,10519.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-08-05,2645.0,27561.0,19661.0,6318.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
modeldata.to_csv('modeldata.csv')